In [ ]:
import compute as cm
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import subprocess

In [ ]:
cwd = os.getcwd()
os.chdir(f"{cwd}/..")

In [ ]:
output = f"{cwd}/../ANALYZE/Analyze.txt"

tinker = "/Users/moseschung/tinker/source"
tinker_analyze = f"{tinker}/analyze.x"
tinker_potential = f"{tinker}/potential.x"
tinker_polarize = f"{tinker}/polarize.x"

monomers = ["ethanol"]

dimersRadial = [
    "ethanol_ethanol",
    "ethanol_WA",
    "ethanol_WD",
]

dimersNRadial = [
    "ethanol_dimer",
]

In [ ]:
QM_monomer = ["potential", "polarizability", "moment"]

QM_dimer = ["MP2", "ALMO", "SAPT"]

index = {
    "ethanol_ethanol": "001",
    "ethanol_WA": "002",
    "ethanol_WD": "003",
    "ethanol_dimer": "004",
}

factorRadial = [0.8, 0.9, 0.95, 1.0, 1.05, 1.1, 1.2]
factorNRadial = [i+1 for i in range(77)]

factorDict = {
    "ethanol_ethanol": factorRadial,
    "ethanol_WA": factorRadial,
    "ethanol_WD": factorRadial,
    "ethanol_dimer": factorNRadial,
}

eqDict = {
    "ethanol_ethanol": [2.811758 * factor for factor in factorRadial],
    "ethanol_WA": [2.886190 * factor for factor in factorRadial],
    "ethanol_WD": [2.803178 * factor for factor in factorRadial],
}

In [ ]:
os.system(f"rm {output}")

f = open(output, "a")
f.close()

In [ ]:
# Potential

os.chdir(f"{cwd}/..")
f = open(output, "a")
f.write("""
#################
##  Potential  ##
#################
""")

if "potential" in QM_monomer:
    os.chdir(f"POTENTIAL")

    for monomer in monomers:
        potentialtext = subprocess.run(
            [tinker_potential, "5", "-k", f"{monomer}.key", f"{monomer}.xyz", f"{monomer}.pot", "N"], 
            capture_output=True, text=True
        ).stdout
        f.write(f"\n # {monomer}\n\n")
        f.write("\n".join(potentialtext.split("\n")[15:]))

f.close()
os.chdir(f"{cwd}/..")

In [ ]:
# Moment

os.chdir(f"{cwd}/..")
f = open(output, "a")
f.write("""
########################
##  Molecular Moment  ##
########################
""")

if "moment" in QM_monomer:
    os.chdir(f"MOMENT")

    for monomer in monomers:
        with open(f"QM_{monomer}_moment.txt", 'r') as file:
            qmtext = file.read()
        qmdip, qmquad = cm.extract_multipole_moments(qmtext)
        mmtext = subprocess.run(
            [tinker_analyze, "-k", f"{monomer}.key", f"{monomer}.xyz", "m"], 
            capture_output=True, text=True
        ).stdout
        mmdip, mmquad = cm.extract_multipole_moments(mmtext)
        diprmsd = cm.calculate_rmsd(qmdip, mmdip)
        quadrmsd = cm.calculate_rmsd(qmquad.flatten(), mmquad.flatten())
        f.write(f"\n # {monomer}\n\n")
        f.write(" QM Dipole X,Y,Z-Components :\n")
        f.write(f"               {qmdip[0]:10.3f}   {qmdip[1]:10.3f}   {qmdip[2]:10.3f}\n")
        f.write(" MM Dipole X,Y,Z-Components :\n")
        f.write(f"               {mmdip[0]:10.3f}   {mmdip[1]:10.3f}   {mmdip[2]:10.3f}\n")
        f.write(" QM Quadrupole Moment Tensor (Buckinghams) :")
        f.write(f"""
                {qmquad[0][0]:10.3f}   {qmquad[0][1]:10.3f}   {qmquad[0][2]:10.3f}
                {qmquad[1][0]:10.3f}   {qmquad[1][1]:10.3f}   {qmquad[1][2]:10.3f}
                {qmquad[2][0]:10.3f}   {qmquad[2][1]:10.3f}   {qmquad[2][2]:10.3f}\n""")
        f.write(" MM Quadrupole Moment Tensor (Buckinghams) :")
        f.write(f"""
                {mmquad[0][0]:10.3f}   {mmquad[0][1]:10.3f}   {mmquad[0][2]:10.3f}
                {mmquad[1][0]:10.3f}   {mmquad[1][1]:10.3f}   {mmquad[1][2]:10.3f}
                {mmquad[2][0]:10.3f}   {mmquad[2][1]:10.3f}   {mmquad[2][2]:10.3f}\n\n""")
        f.write(f" Dipole RMSD :                            {diprmsd:10.3f}\n")
        f.write(f" Quadrupole RMSD :                        {quadrmsd:10.3f}\n")

f.close()
os.chdir(f"{cwd}/..")

In [ ]:
# Polarizability

os.chdir(f"{cwd}/..")
f = open(output, "a")
f.write("""
################################
##  Molecular Polarizability  ##
################################
""")

if "polarizability" in QM_monomer:
    os.chdir(f"POLAR")

    for monomer in monomers:
        with open(f"QM_{monomer}_pol.txt", 'r') as file:
            qmtext = file.read()
        qmten, qmeig, qmpol = cm.extract_polarizability(qmtext)
        mmtext = subprocess.run(
            [tinker_polarize, "-k", f"{monomer}.key", f"{monomer}.xyz"], 
            capture_output=True, text=True
        ).stdout
        mmten, mmeig, mmpol = cm.extract_polarizability(mmtext)
        tenrmsd = cm.calculate_rmsd(qmten.flatten(), mmten.flatten())
        eigrmsd = cm.calculate_rmsd(qmeig, mmeig)
        f.write(f"\n # {monomer}\n\n")
        f.write(" QM Molecular Polarizability Tensor :")
        f.write(f"""
                {qmten[0][0]:10.3f}   {qmten[0][1]:10.3f}   {qmten[0][2]:10.3f}
                {qmten[1][0]:10.3f}   {qmten[1][1]:10.3f}   {qmten[1][2]:10.3f}
                {qmten[2][0]:10.3f}   {qmten[2][1]:10.3f}   {qmten[2][2]:10.3f}\n""")
        f.write(" MM Molecular Polarizability Tensor :")
        f.write(f"""
                {mmten[0][0]:10.3f}   {mmten[0][1]:10.3f}   {mmten[0][2]:10.3f}
                {mmten[1][0]:10.3f}   {mmten[1][1]:10.3f}   {mmten[1][2]:10.3f}
                {mmten[2][0]:10.3f}   {mmten[2][1]:10.3f}   {mmten[2][2]:10.3f}\n\n""")
        f.write(" QM Polarizability Tensor Eigenvalues :\n")
        f.write(f"               {qmeig[0]:10.3f}   {qmeig[1]:10.3f}   {qmeig[2]:10.3f}\n")
        f.write(" MM Polarizability Tensor Eigenvalues :\n")
        f.write(f"               {mmeig[0]:10.3f}   {mmeig[1]:10.3f}   {mmeig[2]:10.3f}\n\n")
        f.write(f" QM Interactive Molecular Polarizability :{qmpol:10.3f}\n")
        f.write(f" MM Interactive Molecular Polarizability :{mmpol:10.3f}\n\n")
        f.write(f" Polarizability Tensor RMSD :             {tenrmsd:10.3f}\n")
        f.write(f" Polarizability Eigenvalues RMSD :        {eigrmsd:10.3f}\n")
        f.write(f" Interactive Polarizability Difference :  {qmpol-mmpol:10.3f}\n")
        

f.close()
os.chdir(f"{cwd}/..")

In [ ]:
# Read in data

os.chdir(f"{cwd}/..")

QM_methods = cm.getQMMethods(QM_dimer)

# Initialize AMOEBA energy

AMOEBA_energy = {
    "Filename": [],
    "Total": [],
    "Electrostatics": [],
    "Induction": [],
    "VdW": [],
}

In [ ]:
# Radial Dimer

os.chdir(f"{cwd}/../XYZ")

for dimer in dimersRadial:
    energy = cm.compute(tinker_analyze, f"{dimer}.key", f"{dimer}.arc", f"{dimer}_A.arc", f"{dimer}_B.arc")
    if len(factorDict[dimer]) != len(energy):
        raise Exception
    for n,factor in enumerate(factorDict[dimer]):
        AMOEBA_energy["Filename"].append(f"{index[dimer]}_{dimer}_{factor:.2f}")
        AMOEBA_energy["Total"].append(energy[n].pot)
        AMOEBA_energy["Electrostatics"].append(energy[n].ele)
        AMOEBA_energy["Induction"].append(energy[n].pol)
        AMOEBA_energy["VdW"].append(energy[n].vdw)

os.chdir(f"{cwd}/..")

In [ ]:
# Nonradial Dimer

os.chdir(f"{cwd}/../XYZ")

for dimer in dimersNRadial:
    energy = cm.compute(tinker_analyze, f"{dimer}.key", f"{dimer}.arc", f"{dimer}_A.arc", f"{dimer}_B.arc")
    if len(factorDict[dimer]) != len(energy):
        raise Exception
    for n,factor in enumerate(factorDict[dimer]):
        AMOEBA_energy["Filename"].append(f"{index[dimer]}_{dimer}_{factor:02d}")
        AMOEBA_energy["Total"].append(energy[n].pot)
        AMOEBA_energy["Electrostatics"].append(energy[n].ele)
        AMOEBA_energy["Induction"].append(energy[n].pol)
        AMOEBA_energy["VdW"].append(energy[n].vdw)

os.chdir(f"{cwd}/..")

In [ ]:
# Convert to dataframe

AMOEBA_energy = pd.DataFrame(AMOEBA_energy)

In [ ]:
# Combine AMOEBA and QM Dataframe

AMOEBA_energy = AMOEBA_energy.rename(columns={"Total": "MM_Tot"})
AMOEBA_energy = AMOEBA_energy.rename(columns={"Electrostatics": "MM_Ele"})
AMOEBA_energy = AMOEBA_energy.rename(columns={"Induction": "MM_Ind"})
AMOEBA_energy = AMOEBA_energy.rename(columns={"VdW": "MM_VdW"})

for method in QM_dimer:
    QM_methods[method] = QM_methods[method].rename(columns={"Total": f"{method}_Tot"})
    QM_methods[method] = QM_methods[method].rename(columns={"Electrostatics": f"{method}_Ele"})
    QM_methods[method] = QM_methods[method].rename(columns={"Induction": f"{method}_Ind"})
    QM_methods[method] = QM_methods[method].rename(columns={"VdW": f"{method}_VdW"})

combined_energy = pd.concat([
    AMOEBA_energy[['Filename']],
    AMOEBA_energy[['MM_Tot']],
    QM_methods['MP2'][['MP2_Tot']],
    QM_methods['ALMO'][['ALMO_Tot']],
    QM_methods['SAPT'][['SAPT_Tot']],
    AMOEBA_energy[['MM_Ele']],
    QM_methods['ALMO'][['ALMO_Ele']],
    QM_methods['SAPT'][['SAPT_Ele']],
    AMOEBA_energy[['MM_Ind']],
    QM_methods['ALMO'][['ALMO_Ind']],
    QM_methods['SAPT'][['SAPT_Ind']],
    AMOEBA_energy[['MM_VdW']],
    QM_methods['ALMO'][['ALMO_VdW']],
    QM_methods['SAPT'][['SAPT_VdW']],
], axis=1)

In [ ]:
# Organize output

# Define the column formats
column_format = {
    "Filename": "{:<25}",
    "MM_Tot": "{:8.3f}",
    "MP2_Tot": "{:8.3f}",
    "ALMO_Tot": "{:8.3f}",
    "SAPT_Tot": "{:8.3f}",
    "MM_Ele": "{:8.3f}",
    "ALMO_Ele": "{:8.3f}",
    "SAPT_Ele": "{:8.3f}",
    "MM_Ind": "{:8.3f}",
    "ALMO_Ind": "{:8.3f}",
    "SAPT_Ind": "{:8.3f}",
    "MM_VdW": "{:8.3f}",
    "ALMO_VdW": "{:8.3f}",
    "SAPT_VdW": "{:8.3f}",
}

# Create the formatted output
formatted_rows = []
header = f"{'Filename':<25} " + " ".join(f"{col:>8}" for col in combined_energy.columns if col != "Filename")
formatted_rows.append(header)

for _, row in combined_energy.iterrows():
    formatted_row = " ".join(column_format[col].format(row[col]) if col in column_format else f"{row[col]:>8}" for col in combined_energy.columns)
    formatted_rows.append(formatted_row)

# Join formatted rows into a single text string
formatted_output = "\n".join(formatted_rows)

In [ ]:
# Compute RMSD

def rmsd(reference, computed):
    return np.sqrt(((computed - reference) ** 2).mean())

def log_rmsd(reference, computed):
    return np.sqrt((np.log1p(np.abs(computed - reference)) ** 2).mean())

MM_MP2_Tot_RMSD = rmsd(combined_energy["MP2_Tot"], combined_energy["MM_Tot"])
MM_ALMO_Tot_RMSD = rmsd(combined_energy["ALMO_Tot"], combined_energy["MM_Tot"])
MM_SAPT_Tot_RMSD = rmsd(combined_energy["SAPT_Tot"], combined_energy["MM_Tot"])
MM_ALMO_Ele_RMSD = rmsd(combined_energy["ALMO_Ele"], combined_energy["MM_Ele"])
MM_SAPT_Ele_RMSD = rmsd(combined_energy["SAPT_Ele"], combined_energy["MM_Ele"])
MM_ALMO_Ind_RMSD = rmsd(combined_energy["ALMO_Ind"], combined_energy["MM_Ind"])
MM_SAPT_Ind_RMSD = rmsd(combined_energy["SAPT_Ind"], combined_energy["MM_Ind"])
MM_ALMO_VdW_RMSD = rmsd(combined_energy["ALMO_VdW"], combined_energy["MM_VdW"])
MM_SAPT_VdW_RMSD = rmsd(combined_energy["SAPT_VdW"], combined_energy["MM_VdW"])
RMSD_txt = [
    f"{'RMSD':<25}",
    f"{'------':>8}",
    f"{MM_MP2_Tot_RMSD:8.3f}",
    f"{MM_ALMO_Tot_RMSD:8.3f}",
    f"{MM_SAPT_Tot_RMSD:8.3f}",
    f"{'------':>8}",
    f"{MM_ALMO_Ele_RMSD:8.3f}",
    f"{MM_SAPT_Ele_RMSD:8.3f}",
    f"{'------':>8}",
    f"{MM_ALMO_Ind_RMSD:8.3f}",
    f"{MM_SAPT_Ind_RMSD:8.3f}",
    f"{'------':>8}",
    f"{MM_ALMO_VdW_RMSD:8.3f}",
    f"{MM_SAPT_VdW_RMSD:8.3f}",
]

MM_MP2_Tot_LRMSD = log_rmsd(combined_energy["MP2_Tot"], combined_energy["MM_Tot"])
MM_ALMO_Tot_LRMSD = log_rmsd(combined_energy["ALMO_Tot"], combined_energy["MM_Tot"])
MM_SAPT_Tot_LRMSD = log_rmsd(combined_energy["SAPT_Tot"], combined_energy["MM_Tot"])
MM_ALMO_Ele_LRMSD = log_rmsd(combined_energy["ALMO_Ele"], combined_energy["MM_Ele"])
MM_SAPT_Ele_LRMSD = log_rmsd(combined_energy["SAPT_Ele"], combined_energy["MM_Ele"])
MM_ALMO_Ind_LRMSD = log_rmsd(combined_energy["ALMO_Ind"], combined_energy["MM_Ind"])
MM_SAPT_Ind_LRMSD = log_rmsd(combined_energy["SAPT_Ind"], combined_energy["MM_Ind"])
MM_ALMO_VdW_LRMSD = log_rmsd(combined_energy["ALMO_VdW"], combined_energy["MM_VdW"])
MM_SAPT_VdW_LRMSD = log_rmsd(combined_energy["SAPT_VdW"], combined_energy["MM_VdW"])
LRMSD_txt = [
    f"{'Log Transformed RMSD':<25}",
    f"{'------':>8}",
    f"{MM_MP2_Tot_LRMSD:8.3f}",
    f"{MM_ALMO_Tot_LRMSD:8.3f}",
    f"{MM_SAPT_Tot_LRMSD:8.3f}",
    f"{'------':>8}",
    f"{MM_ALMO_Ele_LRMSD:8.3f}",
    f"{MM_SAPT_Ele_LRMSD:8.3f}",
    f"{'------':>8}",
    f"{MM_ALMO_Ind_LRMSD:8.3f}",
    f"{MM_SAPT_Ind_LRMSD:8.3f}",
    f"{'------':>8}",
    f"{MM_ALMO_VdW_LRMSD:8.3f}",
    f"{MM_SAPT_VdW_LRMSD:8.3f}",
]

In [ ]:
# Write output

os.chdir(f"{cwd}/..")
f = open(output, "a")
f.write("""
############################
##  Energy Decomposition  ##
############################
""")
f.write("\n")
f.close()

# Write the formatted output to a text file
with open(output, "a") as file:
    file.write(formatted_output)

f = open(output, "a")
f.write("\n\n")
f.write(" ".join(RMSD_txt) + "\n")
f.write(" ".join(LRMSD_txt) + "\n")
f.close()

In [ ]:
# Plot

os.chdir(f"{cwd}/../ANALYZE")

markers = ["o", "s", "^", "d"]

for dimer in dimersRadial:

    filtered_df = combined_energy[combined_energy["Filename"].str.startswith(index[dimer])]
    
    plt.figure(figsize=(9, 5), dpi=100)
    for i, col in enumerate(["MM_Tot", "ALMO_Tot", "SAPT_Tot", "MP2_Tot"]):
        plt.plot(eqDict[dimer], filtered_df[col], marker=markers[i], linestyle="-", label=col)
    plt.axvline(x=eqDict[dimer][3], color="gray", linestyle="dotted", linewidth=2, label="Equilibrium Distance")
    plt.xlabel("O-O Distance (Å)", fontsize=12)
    plt.ylabel("Energy (kcal/mol)", fontsize=12)
    plt.title(f"{dimer.capitalize()} Total Energy", fontsize=14)
    plt.legend(fontsize=10)
    plt.grid(True, which='major', linestyle="--", alpha=0.6)
    plt.grid(True, which='minor', linestyle=":", alpha=0.3)
    plt.xticks(fontsize=10)
    plt.yticks(fontsize=10)
    plt.minorticks_on()
    plt.tick_params(axis='both', which='minor', direction='in', length=4, width=0.8, color='gray')
    plt.savefig(f"{index[dimer]}_{dimer}_Tot.png", dpi=300, bbox_inches="tight")
    plt.show()
    
    plt.figure(figsize=(9, 5), dpi=100)
    for i, col in enumerate(["MM_Ele", "ALMO_Ele", "SAPT_Ele"]):
        plt.plot(eqDict[dimer], filtered_df[col], marker=markers[i], linestyle="-", label=col)
    plt.axvline(x=eqDict[dimer][3], color="gray", linestyle="dotted", linewidth=2, label="Equilibrium Distance")
    plt.xlabel("O-O Distance (Å)", fontsize=12)
    plt.ylabel("Energy (kcal/mol)", fontsize=12)
    plt.title(f"{dimer.capitalize()} Electrostatic Energy", fontsize=14)
    plt.legend(fontsize=10)
    plt.grid(True, which='major', linestyle="--", alpha=0.6)
    plt.grid(True, which='minor', linestyle=":", alpha=0.3)
    plt.xticks(fontsize=10)
    plt.yticks(fontsize=10)
    plt.minorticks_on()
    plt.tick_params(axis='both', which='minor', direction='in', length=4, width=0.8, color='gray')
    plt.savefig(f"{index[dimer]}_{dimer}_Ele.png", dpi=300, bbox_inches="tight")
    plt.show()
    
    plt.figure(figsize=(9, 5), dpi=100)
    for i, col in enumerate(["MM_Ind", "ALMO_Ind", "SAPT_Ind"]):
        plt.plot(eqDict[dimer], filtered_df[col], marker=markers[i], linestyle="-", label=col)
    plt.axvline(x=eqDict[dimer][3], color="gray", linestyle="dotted", linewidth=2, label="Equilibrium Distance")
    plt.xlabel("O-O Distance (Å)", fontsize=12)
    plt.ylabel("Energy (kcal/mol)", fontsize=12)
    plt.title(f"{dimer.capitalize()} Induction Energy", fontsize=14)
    plt.legend(fontsize=10)
    plt.grid(True, which='major', linestyle="--", alpha=0.6)
    plt.grid(True, which='minor', linestyle=":", alpha=0.3)
    plt.xticks(fontsize=10)
    plt.yticks(fontsize=10)
    plt.minorticks_on()
    plt.tick_params(axis='both', which='minor', direction='in', length=4, width=0.8, color='gray')
    plt.savefig(f"{index[dimer]}_{dimer}_Ind.png", dpi=300, bbox_inches="tight")
    plt.show()
    
    plt.figure(figsize=(9, 5), dpi=100)
    for i, col in enumerate(["MM_VdW", "ALMO_VdW", "SAPT_VdW"]):
        plt.plot(eqDict[dimer], filtered_df[col], marker=markers[i], linestyle="-", label=col)
    plt.axvline(x=eqDict[dimer][3], color="gray", linestyle="dotted", linewidth=2, label="Equilibrium Distance")
    plt.xlabel("O-O Distance (Å)", fontsize=12)
    plt.ylabel("Energy (kcal/mol)", fontsize=12)
    plt.title(f"{dimer.capitalize()} VdW Energy", fontsize=14)
    plt.legend(fontsize=10)
    plt.grid(True, which='major', linestyle="--", alpha=0.6)
    plt.grid(True, which='minor', linestyle=":", alpha=0.3)
    plt.xticks(fontsize=10)
    plt.yticks(fontsize=10)
    plt.minorticks_on()
    plt.tick_params(axis='both', which='minor', direction='in', length=4, width=0.8, color='gray')
    plt.savefig(f"{index[dimer]}_{dimer}_VdW.png", dpi=300, bbox_inches="tight")
    plt.show()

os.chdir(f"{cwd}/..")

In [ ]:
# Plot

os.chdir(f"{cwd}/../ANALYZE")

markers = ["o", "s", "^", "d"]

for dimer in dimersNRadial:

    filtered_df = combined_energy[combined_energy["Filename"].str.startswith(index[dimer])]

    plt.figure(figsize=(9, 5), dpi=100)
    for i, col in enumerate(["ALMO_Tot", "SAPT_Tot", "MP2_Tot"]):
        plt.scatter(filtered_df[col], filtered_df["MM_Tot"], marker=markers[i], linestyle="-", label=col)
    minmax = [filtered_df["MP2_Tot"].min(), filtered_df["MP2_Tot"].max()]
    plt.plot(minmax, minmax, c="gray", ls="--")
    plt.xlabel("QM Energy (kcal/mol)", fontsize=12)
    plt.ylabel("MM Energy (kcal/mol)", fontsize=12)
    plt.title(f"{dimer.capitalize()} Total Energy", fontsize=14)
    plt.legend(fontsize=10)
    plt.grid(True, which='major', linestyle="--", alpha=0.6)
    plt.grid(True, which='minor', linestyle=":", alpha=0.3)
    plt.xticks(fontsize=10)
    plt.yticks(fontsize=10)
    plt.minorticks_on()
    plt.tick_params(axis='both', which='minor', direction='in', length=4, width=0.8, color='gray')
    plt.savefig(f"{index[dimer]}_{dimer}_Tot.png", dpi=300, bbox_inches="tight")
    plt.show()

    plt.figure(figsize=(9, 5), dpi=100)
    for i, col in enumerate(["ALMO_Ele", "SAPT_Ele"]):
        plt.scatter(filtered_df[col], filtered_df["MM_Ele"], marker=markers[i], linestyle="-", label=col)
    minmax = [filtered_df["ALMO_Ele"].min(), filtered_df["ALMO_Ele"].max()]
    plt.plot(minmax, minmax, c="gray", ls="--")
    plt.xlabel("QM Energy (kcal/mol)", fontsize=12)
    plt.ylabel("MM Energy (kcal/mol)", fontsize=12)
    plt.title(f"{dimer.capitalize()} Electrostatic Energy", fontsize=14)
    plt.legend(fontsize=10)
    plt.grid(True, which='major', linestyle="--", alpha=0.6)
    plt.grid(True, which='minor', linestyle=":", alpha=0.3)
    plt.xticks(fontsize=10)
    plt.yticks(fontsize=10)
    plt.minorticks_on()
    plt.tick_params(axis='both', which='minor', direction='in', length=4, width=0.8, color='gray')
    plt.savefig(f"{index[dimer]}_{dimer}_Ele.png", dpi=300, bbox_inches="tight")
    plt.show()

    plt.figure(figsize=(9, 5), dpi=100)
    for i, col in enumerate(["ALMO_Ind", "SAPT_Ind"]):
        plt.scatter(filtered_df[col], filtered_df["MM_Ind"], marker=markers[i], linestyle="-", label=col)
    minmax = [filtered_df["ALMO_Ind"].min(), filtered_df["ALMO_Ind"].max()]
    plt.plot(minmax, minmax, c="gray", ls="--")
    plt.xlabel("QM Energy (kcal/mol)", fontsize=12)
    plt.ylabel("MM Energy (kcal/mol)", fontsize=12)
    plt.title(f"{dimer.capitalize()} Induction Energy", fontsize=14)
    plt.legend(fontsize=10)
    plt.grid(True, which='major', linestyle="--", alpha=0.6)
    plt.grid(True, which='minor', linestyle=":", alpha=0.3)
    plt.xticks(fontsize=10)
    plt.yticks(fontsize=10)
    plt.minorticks_on()
    plt.tick_params(axis='both', which='minor', direction='in', length=4, width=0.8, color='gray')
    plt.savefig(f"{index[dimer]}_{dimer}_Ind.png", dpi=300, bbox_inches="tight")
    plt.show()

    plt.figure(figsize=(9, 5), dpi=100)
    for i, col in enumerate(["ALMO_VdW", "SAPT_VdW"]):
        plt.scatter(filtered_df[col], filtered_df["MM_VdW"], marker=markers[i], linestyle="-", label=col)
    minmax = [filtered_df["ALMO_VdW"].min(), filtered_df["ALMO_VdW"].max()]
    plt.plot(minmax, minmax, c="gray", ls="--")
    plt.xlabel("QM Energy (kcal/mol)", fontsize=12)
    plt.ylabel("MM Energy (kcal/mol)", fontsize=12)
    plt.title(f"{dimer.capitalize()} VdW Energy", fontsize=14)
    plt.legend(fontsize=10)
    plt.grid(True, which='major', linestyle="--", alpha=0.6)
    plt.grid(True, which='minor', linestyle=":", alpha=0.3)
    plt.xticks(fontsize=10)
    plt.yticks(fontsize=10)
    plt.minorticks_on()
    plt.tick_params(axis='both', which='minor', direction='in', length=4, width=0.8, color='gray')
    plt.savefig(f"{index[dimer]}_{dimer}_VdW.png", dpi=300, bbox_inches="tight")
    plt.show()